This is a notebook that can be used to pull data from the iRacing API.

https://forums.iracing.com/discussion/15068/general-availability-of-data-api/p1

uses the [iracingdataapi package](https://github.com/jasondilworth56/iracingdataapi) to pull data from the iRacing API.



In [ ]:
!pip install git+https://github.com/jasondilworth56/iracingdataapi.git


In [ ]:
from iracingdataapi.client import irDataClient

import os

username = os.getenv("IRACING_USERNAME")
password = os.getenv("IRACING_PASSWORD")
# alternatively you can set the username and password directly here
# fill in your username and password, an uncomment the lines below
# username = "putyourusernamehereitisusuallyyouremailaddress"
# password = "putyourpasswordhere"

customer_id = 643506 # this is the customer id for getting user stats

def calculate_f499_points(start, finish, incident_count):
    return (start - finish) + (4 if incident_count == 0 else -1 * (2 * incident_count))

def get_session_link(subsession_id, new_ui=False):
    if new_ui:
        format = "https://members-ng.iracing.com/racing/results-stats/results?subsessionid="
    else:
        format = "https://members.iracing.com/membersite/member/EventResult.do?&subsessionid="
    
    return f"{format}{subsession_id})"

idc = irDataClient(username=username, password=password)

# if you do not pass a customer_id as the param to the method call, it will default to the customer_id of the user you are logged in as
stats = idc.stats_member_recent_races() 

# get the stats for the last 10 races
last_10_races = stats['races'][:10]

# iterate over the last 10 races
for i, race in enumerate(last_10_races, start=1):
    start_position = race['start_position']
    finish_position = race['finish_position']
    incident_count = race['incidents']
    points = calculate_f499_points(start_position, finish_position, incident_count)
    
    
    print(f"Race {race['session_start_time']}:")
    print(f"Session Link: {get_session_link(race['subsession_id'], True)}")
    print(f"Series: {race['series_name']}")
    print(f"Track: {race['track']['track_name']}")
    print(f"Start Position: {start_position if start_position else 'N/A'}")
    print(f"Finish Position: {finish_position if finish_position else 'N/A'}")
    print(f"Number of Incidents: {incident_count}")
    print(f"Points: {points}")
    print()